In [0]:
from dlt import *
from pyspark.sql.functions import *

# Create the target bronze table with the required properties
dlt.create_streaming_table(
    name="demo.dbdemos_dlt_cdc.weekly_hospital_respiratory_data_bronze",
    comment="New customer data incrementally ingested from cloud object storage landing zone",
    table_properties={
        'delta.minReaderVersion': '2',
        'delta.minWriterVersion': '5',
        'delta.columnMapping.mode': 'name'
    }
)

# Create an Append Flow to ingest the raw data into the bronze table
@append_flow(
    target="weekly_hospital_respiratory_data_bronze",
    name="weekly_hospital_bronze_ingest_flow"
)
def weekly_hospital_bronze_ingest_flow():
    return (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "csv")
            .option("cloudFiles.inferColumnTypes", "true")
            .load("/Volumes/demo/dbdemos_dlt_cdc/raw_data/Weekly_Hospital_Respiratory_Data/")
    )